In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 10000 
N_bound = 200 

la = np.array([5,np.pi/2])
lb = np.array([-5,0])

traindata = lb+(la-lb)*lhs(2,N_train)

x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_bound = lb[0]+(la[0]-lb[0])*lhs(1,N_bound)
t_bound = lb[1]+(la[1]-lb[1])*lhs(1,N_bound)

x_inside_5 = 5*np.ones_like(t_bound)
x_inside_fu_5 = -5*np.ones_like(t_bound)

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

x_bound = numpy_to_tensor(x_bound, var_name="x_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
t_bound = numpy_to_tensor(t_bound, var_name="t_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

x_inside_5 = numpy_to_tensor(x_inside_5, var_name="x_inside_5", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
x_inside_fu_5 = numpy_to_tensor(x_inside_fu_5, var_name="x_inside_fu_5", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

In [3]:
data = scipy.io.loadmat('./NLS.mat')

x_exact = data['x'].flatten()[:,None]
t_exact = data['tt'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)

In [ ]:
print('x:',x_exact.shape)
print('t:',t_exact.shape)
print('Exact_u:',Exact_u.shape)
print('Exact_v:',Exact_v.shape)
print('Exact_h:',Exact_h.shape)

In [ ]:
T_exact, X_exact = np.meshgrid(t_exact,x_exact)
print('T_exact:',T_exact.shape)
print('X_exact:',X_exact.shape)

In [6]:
T_exact_flatten = T_exact.flatten()[:,None]
X_exact_flatten = X_exact.flatten()[:,None]
data_star = np.hstack((X_exact_flatten,T_exact_flatten))
u_star = Exact_u.flatten()[:,None]
v_star = Exact_v.flatten()[:,None]
h_star = Exact_h.flatten()[:,None]

In [ ]:
data_star.shape

In [ ]:
X_star = numpy_to_tensor(data_star, var_name="X_star", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [ ]:
X_star

In [ ]:
fig, ax = plt.subplots()
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])
ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
ax.set_title('$|h(t, x)|$', fontsize = 10)
h = ax.imshow(Exact_h, interpolation='nearest', cmap='rainbow', 
              extent=[lb[1], la[1], lb[0], la[0]], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.show()

In [ ]:
np.random.seed(5678)
n_test_data = 10000

index_test_data = (lhs(1,n_test_data))*len(data_star)
index_test_data = np.floor(index_test_data).reshape(n_test_data,).astype(int)

test_data = data_star[index_test_data]
test_h = h_star[index_test_data]

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
test_h = numpy_to_tensor(test_h, var_name="test_h", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [13]:
def sech(x_input):
    sech_x = 2/(torch.exp(x_input)+torch.exp(-x_input))
    return sech_x

In [14]:
real_x_0 = torch.cat((2*sech(x_bound),torch.zeros_like(x_bound)),axis = 1)

## Loss_f

In [15]:
def get_loss_f(x_grad, t_grad, PINNs, return_sequence='not'):
    
    output_inside = PINNs(torch.cat((x_grad, t_grad), 1))
    u_inside = output_inside[:,0:1]
    v_inside = output_inside[:,1:2]
    
    u_xx = compute_higher_order_derivatives(u_inside, [x_grad,x_grad])
    v_xx = compute_higher_order_derivatives(v_inside, [x_grad,x_grad])

    u_t = compute_higher_order_derivatives(u_inside, [t_grad])
    v_t = compute_higher_order_derivatives(v_inside, [t_grad])
  
    f_u = -v_t+0.5*u_xx+(u_inside**2+v_inside**2)*u_inside
    f_v = u_t+0.5*v_xx+(u_inside**2+v_inside**2)*v_inside

    if return_sequence=='yes':
        return torch.square(f_u)+torch.square(f_v)
    else:
        return torch.mean(torch.square(f_u))+torch.mean(torch.square(f_v)) 

## loss_bound

In [16]:
def get_loss_bound(bound_x, bound_t, PINNs, return_sequence='not'):
    
    E_x_0 = PINNs1(torch.cat((bound_x,torch.zeros_like(bound_x)),1))
    loss_bound_for_a = torch.mean(torch.square(E_x_0-real_x_0))
    
    E_t_5 = PINNs1(torch.cat((x_inside_5,bound_t),1))
    E_t_fu_5 = PINNs1(torch.cat((x_inside_fu_5,bound_t),1))
        
    loss_bound_for_b = torch.mean(torch.square(E_t_5-E_t_fu_5))
    
    grad_E_5 = compute_higher_order_derivatives(E_t_5, [x_inside_5])
    grad_E_fu_5 = compute_higher_order_derivatives(E_t_fu_5, [x_inside_fu_5])
    loss_bound_for_c = torch.mean(torch.square(grad_E_5-grad_E_fu_5))
    
    loss_bound_value = loss_bound_for_a+loss_bound_for_b+loss_bound_for_c

    return loss_bound_value

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA on GPU")
else:
    device = "cpu"
    print("Using CPU")

In [18]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [19]:
#x t  ; u v
net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[100]*5, 
                                     output_dims=2, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')
PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [20]:
##############PW_L_f###############
# e_k = 0.01
# q_k = 0.00005
# s_k = 0.5
e_k = 0.001
q_k = 0.00005
s_k = 0.7
Ada_CL_L_f = Ada_CL(e_k,q_k,s_k,device)
weight_sequence_L_f = (1/N_train)*np.ones((N_train,1))
##############PW_L_f###############

############## plot L_f list ###############  
plot_loss_f_list = []
X_star_x = X_star[:,0:1]
X_star_t = X_star[:,1:2]
X_star_x.requires_grad_()
X_star_t.requires_grad_()
############## plot L_f list ###############  

########### record list ###########
loss_all_1 = []
loss_all_2 = []
loss_f_1 = []
loss_f_2 = []
loss_b_1 = []
test_loss_1 = []
beta_number_list = []
rho_k_list = []
loss_f_sequence_list = []
weight_sequence_L_f_list = []
plot_heat_list = []
########### record list ###########

In [ ]:
nIter1 = 20000
it = 0

while it<nIter1:

    ############ loss_bound  ############
    loss_bound = get_loss_bound(x_bound, t_bound, PINNs1, return_sequence='not')
    
    ##############PW_L_f###############
    loss_f_sequence = get_loss_f(x_inside,t_inside,PINNs1,return_sequence='yes')
    loss_f,weight_sequence_L_f,rho_k,beta_sequence = Ada_CL_L_f.Training_Scheduler_torch(loss_f_sequence, 
                                                                                         weight_sequence_L_f,
                                                                                         alpha_k_function = 'default',
                                                                                         record = 'yes')
    beta_number_list.append(np.sum(beta_sequence)/N_train)
    rho_k_list.append(rho_k)
    ##############PW_L_f###############    
        
    #####loss PI#######
    loss = loss_bound*2+loss_f    
    
    ##################  test_loss NRMSE  ##################
    pre_h = torch.sqrt(PINNs1(test_data)[:,0:1]**2 + PINNs1(test_data)[:,1:2]**2)
    test_loss = relative_l2_torch(pre_h,test_h)
    ##################  test_loss NRMSE  ##################  
    
    ##############Record###############
    test_loss_1.append(test_loss)    
    loss_all_2.append(loss.item())
    loss_f_2.append(loss_f.item())
    loss_b_1.append(loss_bound.item())   
    ##############Record###############
    
    ##############Record Original###############
    loss_f_original = get_loss_f(x_inside,t_inside,PINNs1,return_sequence='not')
    loss_all_original = loss_f_original+loss_bound*2
    loss_all_1.append(loss_all_original.item())
    loss_f_1.append(loss_f_original.item())                 
    ##############Record Original###############
    
    if it % 1000 == 0:
        plot_loss_f_list.append(get_loss_f(X_star_x,X_star_t,PINNs1,return_sequence='yes').cpu().detach().numpy())
        loss_f_sequence_list.append(loss_f_sequence.cpu().detach().numpy())
        weight_sequence_L_f_list.append(weight_sequence_L_f)
        plot_heat_list.append(PINNs1(X_star).cpu().detach().numpy())
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss) 

    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    it = it + 1   

print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)